In [92]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cve-dataset-with-embedding/merged_cve_data.csv


In [4]:
! pip install xgboost
!pip install lightgbm catboost torch torchvision torchaudio


In [117]:
df = pd.read_csv('/kaggle/input/cve-dataset-with-embedding/merged_cve_data.csv')

In [118]:
df

,CVE_ID,ASSIGNER,Published_Date,Last_Modified_Date,Impact_Score,Access_Vector,Access_Complexity,Configurations,Reference_Data,Year,...,759,760,761,762,763,764,765,766,767,Description
0,CVE-2011-0199,product-security@apple.com,2011-06-24 20:55:00+00:00,2024-02-09 03:18:00+00:00,3.6,NETWORK,MEDIUM,['cpe:2.3:o:apple:mac_os_x_server:*:*:*:*:*:*:...,"['http://support.apple.com/kb/HT4723', 'http:/...",2011,...,-0.007224,-0.015303,0.056557,-0.039082,0.008622,0.022517,-0.040034,0.014473,0.011391,The Certificate Trust Policy component in Appl...
1,CVE-2011-0220,product-security@apple.com,2020-02-05 20:15:00+00:00,2020-02-07 19:24:00+00:00,3.6,LOCAL,LOW,['cpe:2.3:a:apple:bonjour:*:*:*:*:*:*:*:*'],['https://opensource.apple.com/source/mDNSResp...,2020,...,0.016330,0.049928,0.008830,0.000793,0.017700,0.028334,0.005227,0.049708,-0.015749,Apple Bonjour before 2011 allows a crash via a...
2,CVE-2011-0428,cve@mitre.org,2019-10-29 19:15:00+00:00,2019-11-01 14:55:00+00:00,2.7,NETWORK,MEDIUM,['cpe:2.3:a:ikiwiki:ikiwiki:*:*:*:*:*:*:*:*'],['https://security-tracker.debian.org/tracker/...,2019,...,0.016057,-0.002453,0.037362,-0.007217,-0.034629,-0.005441,-0.000510,-0.015185,-0.025799,Cross Site Scripting (XSS) in ikiwiki before 3...
3,CVE-2011-0467,security@opentext.com,2018-06-07 21:29:00+00:00,2023-11-07 02:06:00+00:00,5.9,NETWORK,LOW,['cpe:2.3:a:suse:studio_onsite_appliance:*:*:*...,['https://bugzilla.suse.com/show_bug.cgi?id=67...,2018,...,0.023072,-0.052349,0.029655,-0.052779,-0.018734,-0.022419,-0.001663,0.004668,-0.070693,A vulnerability in the listing of available so...
4,CVE-2011-0469,security@opentext.com,2017-08-17 16:29:00+00:00,2023-11-07 02:06:00+00:00,5.9,NETWORK,LOW,['cpe:2.3:o:suse:opensuse:-:*:*:*:*:*:*:*'],['https://bugzilla.suse.com/show_bug.cgi?id=67...,2017,...,0.008663,0.009789,0.012439,-0.032353,0.007584,-0.005330,0.010918,-0.033823,0.007783,Code injection in openSUSE when running some s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107082,CVE-2008-7315,cve@mitre.org,2017-10-10 16:29:00+00:00,2017-11-03 17:15:00+00:00,5.9,NETWORK,LOW,['cpe:2.3:a:cpan:ui\\:\\:dialog:0.01:*:*:*:*:*...,['https://security-tracker.debian.org/tracker/...,2017,...,0.019123,-0.046788,0.001866,0.024574,-0.024319,-0.030054,0.025092,-0.023828,-0.022433,UI-Dialog 1.09 and earlier allows remote attac...
107083,CVE-2008-7316,security@debian.org,2016-05-02 10:59:00+00:00,2016-05-06 00:54:00+00:00,3.6,LOCAL,LOW,['cpe:2.3:o:linux:linux_kernel:*:*:*:*:*:*:*:*'],['http://git.kernel.org/cgit/linux/kernel/git/...,2016,...,0.014300,-0.012750,-0.036155,-0.043011,0.024458,0.012200,-0.009282,0.023590,-0.018185,mm/filemap.c in the Linux kernel before 2.6.25...
107084,CVE-2008-7319,cve@mitre.org,2017-11-07 21:29:00+00:00,2017-11-29 15:49:00+00:00,5.9,NETWORK,LOW,['cpe:2.3:a:net-ping-external_project:net-ping...,['https://rt.cpan.org/Public/Bug/Display.html?...,2017,...,0.034606,-0.056721,-0.028375,-0.023293,-0.014179,-0.005051,-0.029391,-0.020186,-0.033210,The Net::Ping::External extension through 0.15...
107085,CVE-2008-7320,cve@mitre.org,2018-11-18 19:29:00+00:00,2024-08-07 12:15:00+00:00,5.9,LOCAL,LOW,['cpe:2.3:a:gnome:seahorse:*:*:*:*:*:*:*:*'],['https://www.bountysource.com/issues/3849352-...,2018,...,-0.022035,0.005189,-0.003003,0.003738,0.012156,-0.027635,-0.001326,0.000663,-0.036381,GNOME Seahorse through 3.30 allows physically ...


# Impact_Score

In [8]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Impact_Score']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [9]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Regressor with GPU
xgb_regressor_gpu = xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate XGBoost
y_pred_xgb_gpu = xgb_regressor_gpu.predict(X_test)
mse_xgb_gpu = mean_squared_error(y_test, y_pred_xgb_gpu)
mae_xgb_gpu = mean_absolute_error(y_test, y_pred_xgb_gpu)
r2_xgb_gpu = r2_score(y_test, y_pred_xgb_gpu)

print("XGBoost GPU Metrics:")
print(f"MSE: {mse_xgb_gpu:.4f}, MAE: {mae_xgb_gpu:.4f}, R²: {r2_xgb_gpu:.4f}\n")


XGBoost GPU Metrics:
MSE: 1.1454, MAE: 0.7909, R²: 0.4984



In [11]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Regressor with GPU
lgb_regressor_gpu = lgb.LGBMRegressor(device='gpu')
lgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate LightGBM
y_pred_lgb_gpu = lgb_regressor_gpu.predict(X_test)
mse_lgb_gpu = mean_squared_error(y_test, y_pred_lgb_gpu)
mae_lgb_gpu = mean_absolute_error(y_test, y_pred_lgb_gpu)
r2_lgb_gpu = r2_score(y_test, y_pred_lgb_gpu)

print("LightGBM GPU Metrics:")
print(f"MSE: {mse_lgb_gpu:.4f}, MAE: {mae_lgb_gpu:.4f}, R²: {r2_lgb_gpu:.4f}\n")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 85669, number of used features: 768
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 768 dense feature groups (62.75 MB) transferred to GPU in 0.050672 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 4.379723
LightGBM GPU Metrics:
MSE: 1.1747, MAE: 0.8198, R²: 0.4856



In [12]:
import pickle
# Save the trained model using pickle
model_filename = 'impact_Score_xgb_regressor.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(xgb_regressor_gpu, file)

print(f"Model saved as {model_filename}")

Model saved as impact_Score_xgb_regressor.pkl


In [91]:
import pandas as pd
import pickle
import numpy as np

# Load the model
with open("impact_Score_xgb_regressor.pkl", 'rb') as file:
    loaded_model = pickle.load(file)

# Ensure y_test is a 1D NumPy array or pandas Series
if isinstance(y_test, pd.DataFrame):
    y_test = y_test.squeeze()  # Convert DataFrame to Series if it's a single column

# Select a random row from the test set
random_index = np.random.randint(0, len(X_test))  # Select random index
random_input = X_test.iloc[random_index].values.reshape(1, -1)  # Use .iloc for correct indexing
real_output = y_test.iloc[random_index] if isinstance(y_test, pd.Series) else y_test[random_index]  # Access real output using .iloc

# Predict the output using the loaded model
predicted_output = loaded_model.predict(random_input)

print(f"Real Output: {real_output:.4f}")
print(f"Predicted Output: {predicted_output[0]:.4f}")


Real Output: 2.0000
Predicted Output: 3.4117


# BASE SCORE

In [15]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Base_Score']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [16]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Regressor with GPU
xgb_regressor_gpu = xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate XGBoost
y_pred_xgb_gpu = xgb_regressor_gpu.predict(X_test)
mse_xgb_gpu = mean_squared_error(y_test, y_pred_xgb_gpu)
mae_xgb_gpu = mean_absolute_error(y_test, y_pred_xgb_gpu)
r2_xgb_gpu = r2_score(y_test, y_pred_xgb_gpu)

print("XGBoost GPU Metrics:")
print(f"MSE: {mse_xgb_gpu:.4f}, MAE: {mae_xgb_gpu:.4f}, R²: {r2_xgb_gpu:.4f}\n")


XGBoost GPU Metrics:
MSE: 1.4874, MAE: 0.9358, R²: 0.4681



In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Regressor with GPU
lgb_regressor_gpu = lgb.LGBMRegressor(device='gpu')
lgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate LightGBM
y_pred_lgb_gpu = lgb_regressor_gpu.predict(X_test)
mse_lgb_gpu = mean_squared_error(y_test, y_pred_lgb_gpu)
mae_lgb_gpu = mean_absolute_error(y_test, y_pred_lgb_gpu)
r2_lgb_gpu = r2_score(y_test, y_pred_lgb_gpu)

print("LightGBM GPU Metrics:")
print(f"MSE: {mse_lgb_gpu:.4f}, MAE: {mae_lgb_gpu:.4f}, R²: {r2_lgb_gpu:.4f}\n")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 85669, number of used features: 768
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 768 dense feature groups (62.75 MB) transferred to GPU in 0.051842 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 6.439187
LightGBM GPU Metrics:
MSE: 1.5338, MAE: 0.9703, R²: 0.4515



In [19]:
import pickle
# Save the trained model using pickle
model_filename = 'base_score_xgb_regressor.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(xgb_regressor_gpu, file)

print(f"Model saved as {model_filename}")

Model saved as base_score_xgb_regressor.pkl


In [22]:
import pandas as pd
import pickle
import numpy as np

# Load the model
with open("base_score_xgb_regressor.pkl", 'rb') as file:
    loaded_model = pickle.load(file)

# Ensure y_test has a reset index if necessary
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

# Select a random row from the test set
random_index = np.random.randint(0, len(X_test))  # Select random index
random_input = X_test.iloc[random_index].values.reshape(1, -1)  # Use .iloc and reshape for prediction
real_output = y_test.iloc[random_index]  # Access real output using .iloc

# Predict the output using the loaded model
predicted_output = loaded_model.predict(random_input)

# Display the results
print(f"Real Output: {real_output:.4f}")
print(f"Predicted Output: {predicted_output[0]:.4f}")


Real Output: 7.4500
Predicted Output: 7.8022


# Exploitability Score

In [23]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Exploitability_Score']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [24]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Regressor with GPU
xgb_regressor_gpu = xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate XGBoost
y_pred_xgb_gpu = xgb_regressor_gpu.predict(X_test)
mse_xgb_gpu = mean_squared_error(y_test, y_pred_xgb_gpu)
mae_xgb_gpu = mean_absolute_error(y_test, y_pred_xgb_gpu)
r2_xgb_gpu = r2_score(y_test, y_pred_xgb_gpu)

print("XGBoost GPU Metrics:")
print(f"MSE: {mse_xgb_gpu:.4f}, MAE: {mae_xgb_gpu:.4f}, R²: {r2_xgb_gpu:.4f}\n")


XGBoost GPU Metrics:
MSE: 1.1731, MAE: 0.8116, R²: 0.4526



In [25]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your dataset
# X, y = ... (Load your features and target variable)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Regressor with GPU
lgb_regressor_gpu = lgb.LGBMRegressor(device='gpu')
lgb_regressor_gpu.fit(X_train, y_train)

# Predict and evaluate LightGBM
y_pred_lgb_gpu = lgb_regressor_gpu.predict(X_test)
mse_lgb_gpu = mean_squared_error(y_test, y_pred_lgb_gpu)
mae_lgb_gpu = mean_absolute_error(y_test, y_pred_lgb_gpu)
r2_lgb_gpu = r2_score(y_test, y_pred_lgb_gpu)

print("LightGBM GPU Metrics:")
print(f"MSE: {mse_lgb_gpu:.4f}, MAE: {mae_lgb_gpu:.4f}, R²: {r2_lgb_gpu:.4f}\n")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 85669, number of used features: 768
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 768 dense feature groups (62.75 MB) transferred to GPU in 0.049208 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 5.308553
LightGBM GPU Metrics:
MSE: 1.2120, MAE: 0.8494, R²: 0.4344



In [26]:
# Save the trained model using pickle
model_filename = 'exploitability_Score_xgb_regressor.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(xgb_regressor_gpu, file)

print(f"Model saved as {model_filename}")

Model saved as exploitability_Score_xgb_regressor.pkl


In [29]:
import pandas as pd
import pickle
import numpy as np

# Load the model
with open("exploitability_Score_xgb_regressor.pkl", 'rb') as file:
    loaded_model = pickle.load(file)

# Reset the index of y_test to ensure it matches with X_test
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)  # Reset X_test index to ensure alignment

# Select a random row from the test set
random_index = np.random.randint(0, len(X_test))  # Select random index
random_input = X_test.iloc[random_index].values.reshape(1, -1)  # Use .iloc for positional indexing
real_output = y_test.iloc[random_index]  # Access real output using .iloc

# Predict the output using the loaded model
predicted_output = loaded_model.predict(random_input)

# Display the results
print(f"Real Output: {real_output:.4f}")
print(f"Predicted Output: {predicted_output[0]:.4f}")


Real Output: 6.9500
Predicted Output: 6.5360


# ACCESS COMPLEXITY

In [53]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Access_Complexity']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from cuml.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import cupy as cp

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Save the LabelEncoder as a pickle file
with open("Access_Complexity_label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

# Variables to store the best model and accuracy
best_model = None
best_accuracy = 0
best_model_name = ""

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    global best_model, best_accuracy, best_model_name
    
    # Fit the model
    model.fit(X_train_gpu if model_name != 'CatBoost' else X_train, 
              y_train_gpu if model_name != 'CatBoost' else y_train)

    predictions = model.predict(X_test_gpu if model_name != 'CatBoost' else X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test_gpu, predictions) if model_name != 'CatBoost' else accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Update the best model if the current model has a higher accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name

# Initialize models with GPU support
models = {
    'XGBoost': XGBClassifier(device='cuda', eval_metric='mlogloss', random_state=42),
    'CatBoost': CatBoostClassifier(task_type='GPU', verbose=0, random_state=42),
    'Random Forest (cuML)': cuRF(n_estimators=100),
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the best model as a pickle file
if best_model:
    with open(f'Access_Complexity_best_{best_model_name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Best model ({best_model_name}) saved with accuracy: {best_accuracy:.4f}")


XGBoost Accuracy: 0.8501
XGBoost Mean Squared Error: 0.1671
XGBoost Mean Absolute Error: 0.1557

CatBoost Accuracy: 0.8517
CatBoost Mean Squared Error: 0.1670
CatBoost Mean Absolute Error: 0.1545

Random Forest (cuML) Accuracy: 0.8327
Random Forest (cuML) Mean Squared Error: 0.1809
Random Forest (cuML) Mean Absolute Error: 0.1718

Best model (CatBoost) saved with accuracy: 0.8517


In [55]:
import pickle
import numpy as np

# Load the saved model and label encoder
with open("Access_Complexity_label_encoder.pkl", 'rb') as f:
    label_encoder = pickle.load(f)

with open("Access_Complexity_best_catboost_model.pkl", 'rb') as f:
    best_model = pickle.load(f)

# Select a random sample from the test set
random_index = np.random.randint(0, X_test.shape[0])
random_sample = X_test[random_index:random_index + 1]

# Predict the label for the sample (CatBoost requires data on CPU)
predicted_label_encoded = best_model.predict(random_sample)

# Convert the encoded prediction back to the original label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))

# Get the actual label for the sample
actual_label_encoded = y_test[random_index]
actual_label = label_encoder.inverse_transform([actual_label_encoded])

# Display the actual and predicted values
print(f"Actual label: {actual_label[0]}")
print(f"Predicted label: {predicted_label[0]}")


Actual label: MEDIUM
Predicted label: MEDIUM


# Access Vector

In [56]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Access_Vector']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from cuml.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import cupy as cp

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Save the LabelEncoder as a pickle file
with open("Access_Vector_label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

# Variables to store the best model and accuracy
best_model = None
best_accuracy = 0
best_model_name = ""

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    global best_model, best_accuracy, best_model_name
    
    # Fit the model
    model.fit(X_train_gpu if model_name != 'CatBoost' else X_train, 
              y_train_gpu if model_name != 'CatBoost' else y_train)

    predictions = model.predict(X_test_gpu if model_name != 'CatBoost' else X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test_gpu, predictions) if model_name != 'CatBoost' else accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Update the best model if the current model has a higher accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name

# Initialize models with GPU support
models = {
    'XGBoost': XGBClassifier(device='cuda', eval_metric='mlogloss', random_state=42),
    'CatBoost': CatBoostClassifier(task_type='GPU', verbose=0, random_state=42),
    'Random Forest (cuML)': cuRF(n_estimators=100),
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the best model as a pickle file
if best_model:
    with open(f'accessVector_best_{best_model_name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Best model ({best_model_name}) saved with accuracy: {best_accuracy:.4f}")


XGBoost Accuracy: 0.9133
XGBoost Mean Squared Error: 0.1291
XGBoost Mean Absolute Error: 0.1008

CatBoost Accuracy: 0.9153
CatBoost Mean Squared Error: 0.1271
CatBoost Mean Absolute Error: 0.0988

Random Forest (cuML) Accuracy: 0.8885
Random Forest (cuML) Mean Squared Error: 0.1629
Random Forest (cuML) Mean Absolute Error: 0.1286

Best model (CatBoost) saved with accuracy: 0.9153


In [60]:
import pickle
import numpy as np

# Load the saved model and label encoder
with open("Access_Vector_label_encoder.pkl", 'rb') as f:
    label_encoder = pickle.load(f)

with open("accessVector_best_catboost_model.pkl", 'rb') as f:
    best_model = pickle.load(f)

# Select a random sample from the test set
random_index = np.random.randint(0, X_test.shape[0])
random_sample = X_test[random_index:random_index + 1]

# Predict the label for the sample (CatBoost requires data on CPU)
predicted_label_encoded = best_model.predict(random_sample)

# Convert the encoded prediction back to the original label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))

# Get the actual label for the sample
actual_label_encoded = y_test[random_index]
actual_label = label_encoder.inverse_transform([actual_label_encoded])

# Display the actual and predicted values
print(f"Actual label: {actual_label[0]}")
print(f"Predicted label: {predicted_label[0]}")


Actual label: LOCAL
Predicted label: LOCAL


# Availability Impact

In [61]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Availability_Impact']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from cuml.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import cupy as cp

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Save the LabelEncoder as a pickle file
with open("Availability_Impact_label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

# Variables to store the best model and accuracy
best_model = None
best_accuracy = 0
best_model_name = ""

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    global best_model, best_accuracy, best_model_name
    
    # Fit the model
    model.fit(X_train_gpu if model_name != 'CatBoost' else X_train, 
              y_train_gpu if model_name != 'CatBoost' else y_train)

    predictions = model.predict(X_test_gpu if model_name != 'CatBoost' else X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test_gpu, predictions) if model_name != 'CatBoost' else accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Update the best model if the current model has a higher accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name

# Initialize models with GPU support
models = {
    'XGBoost': XGBClassifier(device='cuda', eval_metric='mlogloss', random_state=42),
    'CatBoost': CatBoostClassifier(task_type='GPU', verbose=0, random_state=42),
    'Random Forest (cuML)': cuRF(n_estimators=100),
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the best model as a pickle file
if best_model:
    with open(f'Availability_Impact_best_{best_model_name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Best model ({best_model_name}) saved with accuracy: {best_accuracy:.4f}")


XGBoost Accuracy: 0.8713
XGBoost Mean Squared Error: 0.4592
XGBoost Mean Absolute Error: 0.2389

CatBoost Accuracy: 0.8729
CatBoost Mean Squared Error: 0.4511
CatBoost Mean Absolute Error: 0.2351

Random Forest (cuML) Accuracy: 0.8474
Random Forest (cuML) Mean Squared Error: 0.5488
Random Forest (cuML) Mean Absolute Error: 0.2847

Best model (CatBoost) saved with accuracy: 0.8729


In [66]:
import pickle
import numpy as np

# Load the saved model and label encoder
with open("Availability_Impact_label_encoder.pkl", 'rb') as f:
    label_encoder = pickle.load(f)

with open("Availability_Impact_best_catboost_model.pkl", 'rb') as f:
    best_model = pickle.load(f)

# Select a random sample from the test set
random_index = np.random.randint(0, X_test.shape[0])
random_sample = X_test[random_index:random_index + 1]

# Predict the label for the sample (CatBoost requires data on CPU)
predicted_label_encoded = best_model.predict(random_sample)

# Convert the encoded prediction back to the original label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))

# Get the actual label for the sample
actual_label_encoded = y_test[random_index]
actual_label = label_encoder.inverse_transform([actual_label_encoded])

# Display the actual and predicted values
print(f"Actual label: {actual_label[0]}")
print(f"Predicted label: {predicted_label[0]}")


Actual label: HIGH
Predicted label: HIGH


# Confidentiality Impact

In [68]:

import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Confidentiality_Impact']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from cuml.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import cupy as cp

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Save the LabelEncoder as a pickle file
with open("Confidentiality_Impact_label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

# Variables to store the best model and accuracy
best_model = None
best_accuracy = 0
best_model_name = ""

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    global best_model, best_accuracy, best_model_name
    
    # Fit the model
    model.fit(X_train_gpu if model_name != 'CatBoost' else X_train, 
              y_train_gpu if model_name != 'CatBoost' else y_train)

    predictions = model.predict(X_test_gpu if model_name != 'CatBoost' else X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test_gpu, predictions) if model_name != 'CatBoost' else accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Update the best model if the current model has a higher accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name

# Initialize models with GPU support
models = {
    'XGBoost': XGBClassifier(device='cuda', eval_metric='mlogloss', random_state=42),
    'CatBoost': CatBoostClassifier(task_type='GPU', verbose=0, random_state=42),
    'Random Forest (cuML)': cuRF(n_estimators=100),
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the best model as a pickle file
if best_model:
    with open(f'Confidentiality_Impact_best_{best_model_name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Best model ({best_model_name}) saved with accuracy: {best_accuracy:.4f}")


XGBoost Accuracy: 0.8372
XGBoost Mean Squared Error: 0.4241
XGBoost Mean Absolute Error: 0.2499

CatBoost Accuracy: 0.8388
CatBoost Mean Squared Error: 0.4199
CatBoost Mean Absolute Error: 0.2475

Random Forest (cuML) Accuracy: 0.7983
Random Forest (cuML) Mean Squared Error: 0.5488
Random Forest (cuML) Mean Absolute Error: 0.3174

Best model (CatBoost) saved with accuracy: 0.8388


In [71]:
import pickle
import numpy as np

# Load the saved model and label encoder
with open("Confidentiality_Impact_label_encoder.pkl", 'rb') as f:
    label_encoder = pickle.load(f)

with open("Confidentiality_Impact_best_catboost_model.pkl", 'rb') as f:
    best_model = pickle.load(f)

# Select a random sample from the test set
random_index = np.random.randint(0, X_test.shape[0])
random_sample = X_test[random_index:random_index + 1]

# Predict the label for the sample (CatBoost requires data on CPU)
predicted_label_encoded = best_model.predict(random_sample)

# Convert the encoded prediction back to the original label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))

# Get the actual label for the sample
actual_label_encoded = y_test[random_index]
actual_label = label_encoder.inverse_transform([actual_label_encoded])

# Display the actual and predicted values
print(f"Actual label: {actual_label[0]}")
print(f"Predicted label: {predicted_label[0]}")


Actual label: HIGH
Predicted label: HIGH


# Integrity Impact

In [72]:
import pandas as pd

# Assuming 'df' is your DataFrame with all the columns
# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# Separate the target variable
y = df['Integrity_Impact']  # Your target variable

# Select all columns except the excluded ones
X = df.drop(columns=columns_to_exclude)

# Check shapes
print(X.shape)  # Should show (n_samples, n_features) where n_features should only include your embedding columns
print(y.shape)  # Should show (n_samples,)


(107087, 768)
(107087,)


In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from cuml.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import cupy as cp

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Save the LabelEncoder as a pickle file
with open("Integrity_Impactt_label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

# Variables to store the best model and accuracy
best_model = None
best_accuracy = 0
best_model_name = ""

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    global best_model, best_accuracy, best_model_name
    
    # Fit the model
    model.fit(X_train_gpu if model_name != 'CatBoost' else X_train, 
              y_train_gpu if model_name != 'CatBoost' else y_train)

    predictions = model.predict(X_test_gpu if model_name != 'CatBoost' else X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test_gpu, predictions) if model_name != 'CatBoost' else accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test_gpu, predictions) if model_name != 'CatBoost' else mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Update the best model if the current model has a higher accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name

# Initialize models with GPU support
models = {
    'XGBoost': XGBClassifier(device='cuda', eval_metric='mlogloss', random_state=42),
    'CatBoost': CatBoostClassifier(task_type='GPU', verbose=0, random_state=42),
    'Random Forest (cuML)': cuRF(n_estimators=100),
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the best model as a pickle file
if best_model:
    with open(f'Integrity_Impact_best_{best_model_name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Best model ({best_model_name}) saved with accuracy: {best_accuracy:.4f}")


XGBoost Accuracy: 0.8418
XGBoost Mean Squared Error: 0.4629
XGBoost Mean Absolute Error: 0.2598

CatBoost Accuracy: 0.8434
CatBoost Mean Squared Error: 0.4506
CatBoost Mean Absolute Error: 0.2546

Random Forest (cuML) Accuracy: 0.8056
Random Forest (cuML) Mean Squared Error: 0.5835
Random Forest (cuML) Mean Absolute Error: 0.3241

Best model (CatBoost) saved with accuracy: 0.8434


In [75]:
import pickle
import numpy as np

# Load the saved model and label encoder
with open("Integrity_Impactt_label_encoder.pkl", 'rb') as f:
    label_encoder = pickle.load(f)

with open("Integrity_Impact_best_catboost_model.pkl", 'rb') as f:
    best_model = pickle.load(f)

# Select a random sample from the test set
random_index = np.random.randint(0, X_test.shape[0])
random_sample = X_test[random_index:random_index + 1]

# Predict the label for the sample (CatBoost requires data on CPU)
predicted_label_encoded = best_model.predict(random_sample)

# Convert the encoded prediction back to the original label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))

# Get the actual label for the sample
actual_label_encoded = y_test[random_index]
actual_label = label_encoder.inverse_transform([actual_label_encoded])

# Display the actual and predicted values
print(f"Actual label: {actual_label[0]}")
print(f"Predicted label: {predicted_label[0]}")


Actual label: NONE
Predicted label: NONE


# Predicting Vulnerability Impact Metrics

In [123]:
import pandas as pd
import pickle
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Suppress warnings
warnings.filterwarnings("ignore")

# Load your DataFrame
# df = pd.read_csv('your_dataset.csv')  # Uncomment and specify your DataFrame source

# Define columns to exclude
columns_to_exclude = [
    'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
    'Published_Date', 'Last_Modified_Date', 
    'Access_Vector', 'Access_Complexity', 
    'Configurations', 'Reference_Data',
    'Year', 'Base_Score', 'Exploitability_Score', 
    'Confidentiality_Impact', 'Integrity_Impact', 
    'Availability_Impact'
]

# List of target variables
target_variables = [
    'Impact_Score',
    'Base_Score',
    'Exploitability_Score',
    'Access_Complexity',
    'Access_Vector',
    'Availability_Impact',
    'Confidentiality_Impact',
    'Integrity_Impact'
]

# Function to split dataset and return train-test splits
def train_test_split_for_target(target):
    # Separate the target variable
    y = df[target]
    
    # Select all columns except the excluded ones
    X = df.drop(columns=columns_to_exclude)
    
    # If the target is categorical, encode it
    if target in ['Access_Complexity', 'Access_Vector', 'Availability_Impact', 'Confidentiality_Impact', 'Integrity_Impact']:
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
        return train_test_split(X, y, test_size=0.2, random_state=42), label_encoder  # Return encoder for later use
    else:
        return train_test_split(X, y, test_size=0.2, random_state=42), None

# Load regression models
regression_models = {
    "Impact_Score": "impact_Score_xgb_regressor.pkl",
    "Base_Score": "base_score_xgb_regressor.pkl",
    "Exploitability_Score": "exploitability_Score_xgb_regressor.pkl"
}

# Load classification models and label encoders
classification_models = {
    "Access_Complexity": ("Access_Complexity_best_catboost_model.pkl", "Access_Complexity_label_encoder.pkl"),
    "Access_Vector": ("accessVector_best_catboost_model.pkl", "Access_Vector_label_encoder.pkl"),
    "Availability_Impact": ("Availability_Impact_best_catboost_model.pkl", "Availability_Impact_label_encoder.pkl"),
    "Confidentiality_Impact": ("Confidentiality_Impact_best_catboost_model.pkl", "Confidentiality_Impact_label_encoder.pkl"),
    "Integrity_Impact": ("Integrity_Impact_best_catboost_model.pkl", "Integrity_Impactt_label_encoder.pkl"),
}

# Initialize dictionaries to hold actual and predicted values
actual_outputs = {}
predicted_outputs = {}

# Loop through all target variables to get train-test splits and make predictions
for target in target_variables:
    (X_train, X_test, y_train, y_test), label_encoder = train_test_split_for_target(target)

    # Reset index of y_test and X_test if necessary
    y_test = pd.Series(y_test).reset_index(drop=True)
    X_test = pd.DataFrame(X_test).reset_index(drop=True)

    # Select a random row from the test set
    random_index = np.random.randint(0, len(X_test))
    random_input = X_test.iloc[random_index].values.reshape(1, -1)

    # Predict for regression models
    if target in regression_models.keys():
        model_file = regression_models[target]
        with open(model_file, 'rb') as file:
            loaded_model = pickle.load(file)
        real_output = y_test.iloc[random_index]
        predicted_output = loaded_model.predict(random_input)
        actual_outputs[target] = real_output
        predicted_outputs[target] = predicted_output[0]

    # Predict for classification models
    elif target in classification_models.keys():
        model_file, encoder_file = classification_models[target]
        with open(encoder_file, 'rb') as f:
            label_encoder = pickle.load(f)
        with open(model_file, 'rb') as f:
            best_model = pickle.load(f)

        predicted_label_encoded = best_model.predict(random_input)
        predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))
        actual_label_encoded = y_test.iloc[random_index]
        actual_label = label_encoder.inverse_transform([actual_label_encoded])

        actual_outputs[target] = actual_label[0]
        predicted_outputs[target] = predicted_label[0]

# Display actual and predicted values for all outputs
print("Actual Outputs:")
for key, value in actual_outputs.items():
    print(f"{key}: {value}")

print("\nPredicted Outputs:")
for key, value in predicted_outputs.items():
    print(f"{key}: {value}")


Actual Outputs:
Impact_Score: 3.6
Base_Score: 8.65
Exploitability_Score: 5.4
Access_Complexity: MEDIUM
Access_Vector: NETWORK
Availability_Impact: NONE
Confidentiality_Impact: HIGH
Integrity_Impact: NONE

Predicted Outputs:
Impact_Score: 3.181267499923706
Base_Score: 7.35125207901001
Exploitability_Score: 5.7252655029296875
Access_Complexity: LOW
Access_Vector: NETWORK
Availability_Impact: NONE
Confidentiality_Impact: HIGH
Integrity_Impact: NONE
